За бенчмарк был взят SentEval (к сожалению поздно нашел что есть SetnRvalRu, взял бы его раз уж диплом про русский енкодер). Взял word2vec, bert, sentenceTransformer и решил сравнить их между собой по тому какой коэффицент Pearson они выдают для датасетов из SentEval. Для проверки был взят semantic textual similarity STS16. Поскольку будем использовать коэффициент корреляции Person, то нулевую гипотезу можно выдвинуть такую:

$H_0$: два набора эмбеддингов не коррелируют между собой.
То есть наша модель выдает плохие эмбеддинги которые не похожи на те с которыми сверяем.

$H_1:$ иначе.



In [ ]:
!pip install git+https://github.com/facebookresearch/SentEval.git
!pip install transformers

In [ ]:
!git clone https://github.com/facebookresearch/SentEval.git
!cd SentEval/data/downstream && ./get_transfer_data.bash

In [ ]:
!pip install -U sentence-transformers

In [2]:
import numpy as np
import torch
import senteval

In [3]:
import gensim.downloader as api

word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Решил взять небольшого берта чтобы недолго вычислялось

In [16]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')


In [21]:
from sentence_transformers import SentenceTransformer

sent_trans_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [4]:
def prepare(params, samples):
    pass

def w2v_batcher(params, batch):
    embeddings = []

    for sent in batch:
        sent_vecs = [word2vec[word] for word in sent if word in word2vec]
        if not sent_vecs:
            vec = np.zeros(300)  # размерность Word2Vec из Google News
        else:
            vec = np.mean(sent_vecs, axis=0)
        embeddings.append(vec)

    return np.vstack(embeddings)


def bert_batcher(params, batch):
    embeddings = []
    for sent in batch:
        inputs = tokenizer(' '.join(sent), return_tensors='pt', padding='max_length', max_length=64, truncation=True)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        vec = outputs['last_hidden_state'][:, 0, :].squeeze().numpy()
        embeddings.append(vec)
    return np.vstack(embeddings)

def sf_batcher(params, batch):
  sentences = [' '.join(s) for s in batch]
  embeddings = sent_trans_model.encode(sentences)
  return embeddings


Опять де для быстроты вычислений брал только один STS16, так не очень полная картина. но добавить эти вычисления в уже готовый пайплайн неслоэно.

In [12]:
def evaluate_senteval_all(batcher):
    params = {
        'task_path': './SentEval/data/',
        'usepytorch': True,
        'kfold': 5
    }
    # params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 256, 'tenacity': 3, 'epoch_size': 2}
    params['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 128, 'tenacity': 5, 'epoch_size': 4}

    se = senteval.engine.SE(params, batcher, prepare)
    transfer_tasks = ['STS16']
    results = []
    for t in transfer_tasks:
      results.append((t, se.eval(t)))
    return results

In [22]:
answer = []
for batcher in [w2v_batcher, bert_batcher, sf_batcher]:
  res = evaluate_senteval_all(batcher)
  answer.append(res)

/usr/local/lib/python3.10/dist-packages/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/usr/local/lib/python3.10/dist-packages/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


In [27]:
for a in answer:
  print(a[0][0], a[0][1]['all'])

STS16 {'pearson': {'mean': 0.6452153625976156, 'wmean': 0.6423912641208951}, 'spearman': {'mean': 0.6628367265439031, 'wmean': 0.6603904415819786}}
STS16 {'pearson': {'mean': 0.5309356966949423, 'wmean': 0.5354579021823227}, 'spearman': {'mean': 0.5796140222498292, 'wmean': 0.5850813434614706}}
STS16 {'pearson': {'mean': 0.8071611961850509, 'wmean': 0.807255050758997}, 'spearman': {'mean': 0.8212384835354316, 'wmean': 0.8211078325335494}}


Для наглядности будем брать $\textit{all}$. Если коэффицент Pearson высок (близок к 1), это значит, что эмбеддинги хорошо передают семантическую информацию, наши эмбеддинги ближе к тем с которыми сравниваем.
Так же смотрим на p-value. Если оно меньше крит уровня значимости, то отвергаем нулевую гипотезу. Такми образом, мы получили некий пайплайн для сравнения качества эмбеддингов разных моделей.

In [23]:
for a in answer:
  print(a[0][0], a[0][1])

STS16 {'answer-answer': {'pearson': PearsonRResult(statistic=0.43059116806902975, pvalue=6.885515435191658e-13), 'spearman': SignificanceResult(statistic=0.4438652153790475, pvalue=1.0962068061221027e-13), 'nsamples': 254}, 'headlines': {'pearson': PearsonRResult(statistic=0.666399952223126, pvalue=2.440174144879004e-33), 'spearman': SignificanceResult(statistic=0.6787408818306457, pvalue=5.669724531092987e-35), 'nsamples': 249}, 'plagiarism': {'pearson': PearsonRResult(statistic=0.7581914618856052, pvalue=3.123496312156598e-44), 'spearman': SignificanceResult(statistic=0.7527228654254128, pvalue=2.8229994656064166e-43), 'nsamples': 230}, 'postediting': {'pearson': PearsonRResult(statistic=0.7334675423212708, pvalue=1.8503390387515724e-42), 'spearman': SignificanceResult(statistic=0.7893695525323239, pvalue=3.4197764021697353e-53), 'nsamples': 244}, 'question-question': {'pearson': PearsonRResult(statistic=0.6374266884890456, pvalue=3.1567224867796926e-25), 'spearman': SignificanceResu

Как можно видеть для всех задач у нас очень маленькое p-value, для SentenceTransformer оно наименьшее, а коэффициент корреляции наибольший.